Install required libraries

In [1]:
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install torchaudio
!pip install librosa
!pip install jiwer

Log into HuggingFace to store checkpoints in your repository

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/jw4169/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [25]:
!sudo apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.7.1-1+deb10u1).
The following package was automatically installed and is no longer required:
  python3-crcmod
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


Load train/test/val data

In [26]:
from datasets import load_dataset, load_metric, Audio

fleurs_train = load_dataset("google/fleurs", "ko_kr", split="train")
fleurs_test = load_dataset("google/fleurs", "ko_kr", split="test")
fleurs_val = load_dataset("google/fleurs", "ko_kr", split="validation")

Inspect data

In [27]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(fleurs_train)

,id,num_samples,path,audio,transcription,raw_transcription,gender,lang_id,language,lang_group_id
0,274,237120,/home/jw4169/.cache/huggingface/datasets/downloads/extracted/cd469757ec9591112c1038d5154ba1cdfdf4739c4b2c6afa72161671e2a4d204/ko_kr/audio/train/15472103231789556112.wav,"{'path': '15472103231789556112.wav', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.960464477539063e-08, 5.960464477539063e-08, 1.1920928955078125e-07, 5.960464477539063e-08, 1.7881393432617188e-07, 1.1920928955078125e-07, 1.7881393432617188e-07, 1.7881393432617188e-07, 2.384185791015625e-07, 1.7881393432617188e-07, 1.7881393432617188e-07, 1.1920928955078125e-07, 1.1920928955078125e-07, 0.0, -5.960464477539063e-08, -1.1920928955078125e-07, -2.384185791015625e-07, -2.384185791015625e-07, -4.172325134277344e-07, -3.5762786865234375e-07, -5.364418029785156e-07, -4.172325134277344e-07, -5.960464477539062e-07, -3.5762786865234375e-07, -5.364418029785156e-07, -2.980232238769531e-07, -4.172325134277344e-07, -1.1920928955078125e-07, -2.384185791015625e-07, 1.1920928955078125e-07, -5.960464477539063e-08, 2.980232238769531e-07, 1.1920928955078125e-07, 4.76837158203125e-07, 2.384185791015625e-07, 3.5762786865234375e-07, 0.0, -5.960464477539063e-08, -4.172325134277344e-07, 4.76837158203125e-07, 1.430511474609375e-06, -1.3113021850585938e-06, 1.4901161193847656e-06, -1.0132789611816406e-06, -1.0132789611816406e-06, 2.562999725341797e-06, 2.980232238769531e-07, -1.3113021850585938e-06, 2.86102294921875e-06, 2.384185791015625e-07, -5.960464477539063e-08, -4.76837158203125e-07, ...], 'sampling_rate': 16000}",독감 증상이 경미할 경우 환자들이 아무렇지 않게 일상생활을 하면서 더 많은 사람을 계속 감염시킬 수 있다는 우려가 있다,독감 증상이 경미할 경우 환자들이 아무렇지 않게 일상생활을 하면서 더 많은 사람을 계속 감염시킬 수 있다는 우려가 있다.,0,48,Korean,6
1,848,134400,/home/jw4169/.cache/huggingface/datasets/downloads/extracted/cd469757ec9591112c1038d5154ba1cdfdf4739c4b2c6afa72161671e2a4d204/ko_kr/audio/train/6380922622707157877.wav,"{'path': '6380922622707157877.wav', 'array': [1.1920928955078125e-07, -1.7881393432617188e-07, 1.7881393432617188e-07, -1.7881393432617188e-07, 1.7881393432617188e-07, -2.384185791015625e-07, 2.384185791015625e-07, -2.980232238769531e-07, 2.980232238769531e-07, -3.5762786865234375e-07, 4.172325134277344e-07, -4.76837158203125e-07, 5.960464477539062e-07, 1.055002212524414e-05, 2.390146255493164e-05, 3.075599670410156e-05, 2.205371856689453e-05, 3.057718276977539e-05, 2.849102020263672e-05, 2.9742717742919922e-05, 2.682209014892578e-05, 2.5033950805664062e-05, 4.2319297790527344e-05, 1.8894672393798828e-05, 4.112720489501953e-06, 1.901388168334961e-05, 2.6047229766845703e-05, 9.655952453613281e-06, -7.271766662597656e-06, -1.33514404296875e-05, -6.794929504394531e-06, 5.7220458984375e-06, -1.436471939086914e-05, -3.6954879760742188e-06, 1.4066696166992188e-05, -5.245208740234375e-06, -1.2814998626708984e-05, -1.919269561767578e-05, -7.510185241699219e-06, -1.6927719116210938e-05, -1.5676021575927734e-05, 3.635883331298828e-06, 5.304813385009766e-06, 2.014636993408203e-05, 9.655952453613281e-06, 1.0132789611816406e-06, -1.0132789611816406e-06, 7.867813110351562e-06, 7.152557373046875e-07, -2.3305416107177734e-05, -2.8073787689208984e-05, -4.595518112182617e-05, -3.337860107421875e-05, -4.4286251068115234e-05, -6.306171417236328e-05, -4.0411949157714844e-05, -2.9265880584716797e-05, -1.6391277313232422e-05, -2.2470951080322266e-05, -6.9141387939453125e-06, -3.0994415283203125e-06, -1.9073486328125e-05, -1.6689300537109375e-05, -3.075599670410156e-05, -1.633167266845703e-05, -6.794929504394531e-06, -2.7418136596679688e-06, -1.0609626770019531e-05, -2.491474151611328e-05, -2.300739288330078e-05, -3.3974647521972656e-05, -3.898143768310547e-05, -5.030632019042969e-05, -3.898143768310547e-05, -2.4437904357910156e-05, -2.0503997802734375e-05, -1.6

In [28]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).lower()
    return batch

Create vocab.json file

In [29]:
vocab_dict = {"\ubccd": 0, "\uce60": 1, "\uae4a": 2, "\ubb54": 3, "\ub7ec": 4, "\ub974": 5, "\ud280": 6, "\uccc7": 7, "\ub540": 8, "\ud514": 9, "\ubc0c": 10, "\uc881": 11, "\ucc27": 12, "\ubbac": 13, "\ud588": 14, "\uc5f0": 15, "\ud575": 16, "\ubd90": 17, "\ubd07": 18, "\uad81": 19, "\ub738": 20, "\ub10c": 21, "\uc816": 22, "\ub9cf": 23, "\ubc9a": 24, "\ub77d": 25, "\uac00": 26, "\ub86d": 27, "\ub2ec": 28, "\uc290": 29, "\ucef8": 30, "\uc74d": 31, "\uc0c9": 32, "\ub9db": 33, "\ub2e6": 34, "\uae30": 35, "\uc545": 36, "\ubed7": 37, "\ud305": 38, "\ub057": 39, "\uae5d": 40, "\ud6c4": 41, "\uc18c": 42, "\ub07d": 43, "\uc870": 44, "\uacaa": 45, "\ucf54": 46, "\uad11": 47, "\ucee8": 48, "\uc62c": 49, "\ud070": 50, "\ub378": 51, "\uad49": 52, "\uad50": 53, "\ubc1c": 54, "\ub78d": 55, "\ub09c": 56, "\ub9f9": 57, "\ud0a8": 58, "\uc635": 59, "\uc0ad": 60, "\ub179": 61, "\uc5c4": 62, "\ub9c8": 63, "\ucac4": 64, "\ucf5c": 65, "\ub125": 66, "\uc6f9": 67, "\ubed4": 68, "\ubfd0": 69, "\uc5e0": 70, "\ud0dc": 71, "\ud5dd": 72, "\uba55": 73, "\uad2d": 74, "\uba67": 75, "\ud540": 76, "\ub0c9": 77, "\ub35f": 78, "\ub355": 79, "\ub52a": 80, "\ub137": 81, "\ud5a5": 82, "\uc65c": 83, "\ubaa8": 84, "\ud611": 85, "\uc694": 86, "\uad88": 87, "\ubc11": 88, "\uacb0": 89, "\ub364": 90, "\ud329": 91, "\ubed1": 92, "\uc529": 93, "\ud154": 94, "\ubc0f": 95, "\ud380": 96, "\ubc85": 97, "\ucf00": 98, "\uc7a0": 99, "\ub0bc": 100, "\ud321": 101, "\ub9bd": 102, "\ub2f5": 103, "\uac83": 104, "\uad7f": 105, "\ud5c8": 106, "\uc0cc": 107, "\uc751": 108, "\uae43": 109, "\ub290": 110, "\uc644": 111, "\ud0e0": 112, "\ub5a0": 113, "\uac31": 114, "\ud3ed": 115, "\uc954": 116, "\ud587": 117, "\uc6cc": 118, "\ub123": 119, "\uc9d0": 120, "\uac9f": 121, "\uac16": 122, "\ub0b3": 123, "\ub9ce": 124, "\uadfc": 125, "\ud2f4": 126, "\uc774": 127, "\uc4f4": 128, "\ubb3b": 129, "\ud330": 130, "\ubd05": 131, "\ub458": 132, "\ud33b": 133, "\ub7b4": 134, "\ub4b7": 135, "\uc98c": 136, "\ud615": 137, "\ud6fc": 138, "\ubb36": 139, "\uc500": 140, "\uafe8": 141, "\ud0c1": 142, "\uc15c": 143, "\uba4b": 144, "\ubd81": 145, "\ub25c": 146, "\ub8f0": 147, "\ud658": 148, "\ud6cc": 149, "\ub809": 150, "\ube0c": 151, "\ud2bc": 152, "\uadc0": 153, "\ud589": 154, "\ucf58": 155, "\uc5c8": 156, "\uc0bc": 157, "\uacbd": 158, "\uc0f5": 159, "\uc704": 160, "\ub2db": 161, "\uc26c": 162, "\ubbfc": 163, "\ub4a4": 164, "\uc170": 165, "\uc11c": 166, "\uad6d": 167, "\ud45c": 168, "\ube4c": 169, "\ub799": 170, "\ub370": 171, "\uace4": 172, "\uc744": 173, "\ubd84": 174, "\ud3c4": 175, "\uc9f8": 176, "\ud15c": 177, "\ubc00": 178, "\uada4": 179, "\ucf69": 180, "\ub1e8": 181, "\ubc24": 182, "\ucef5": 183, "\uc775": 184, "\uc62e": 185, "\ucd9c": 186, "\uc90d": 187, "\uc9d6": 188, "\ub610": 189, "\ucf11": 190, "\uce6d": 191, "\ubcf8": 192, "\ub959": 193, "\uc791": 194, "\uafce": 195, "\uc571": 196, "\ub9fa": 197, "\ub8e8": 198, "\ud53d": 199, "\ub358": 200, "\uc19c": 201, "\ub534": 202, "\ube48": 203, "\uc96c": 204, "\uc796": 205, "\ucc22": 206, "\ub301": 207, "\ubd24": 208, "\ud17c": 209, "\ub9bf": 210, "\ub7b5": 211, "\uad73": 212, "\uae6c": 213, "\ub2ac": 214, "\ub8f8": 215, "\uc6b4": 216, "\ubcb3": 217, "\ucdb0": 218, "\ub6f4": 219, "\ub985": 220, "\uc058": 221, "\ud150": 222, "\ud798": 223, "\uc608": 224, "\uc61b": 225, "\ud5e4": 226, "\ub294": 227, "\ub625": 228, "\ud0c8": 229, "\ubc31": 230, "\ud0a5": 231, "\ubb3c": 232, "\uc787": 233, "\uaed8": 234, "\uce6b": 235, "\uc11d": 236, "\ub80c": 237, "\ucda9": 238, "\uc7a1": 239, "\ub2dd": 240, "\ub04c": 241, "\ud23c": 242, "\uc804": 243, "\ub17c": 244, "\ucc44": 245, "\ud2b8": 246, "\ubcd5": 247, "\ud0b7": 248, "\ub0b8": 249, "\ube45": 250, "\ub801": 251, "\ub188": 252, "\ub0a8": 253, "\ubd80": 254, "\uac1a": 255, "\uc464": 256, "\ub93c": 257, "\ud3c8": 258, "\uceeb": 259, "\ubcd1": 260, "\uace0": 261, "\ubc15": 262, "\ud669": 263, "\ucf64": 264, "\ub2e5": 265, "\ud5d0": 266, "\ub2a6": 267, "\uc735": 268, "\ud2f1": 269, "\ucc38": 270, "\ud2c8": 271, "\ub764": 272, "\uc12d": 273, "\ub208": 274, "\ud76c": 275, "\uc7a5": 276, "\ubc30": 277, "\ud574": 278, "\ud5f4": 279, "\uc9ca": 280, "\ubab0": 281, "\ub180": 282, "\ubd89": 283, "\uce7c": 284, "\ub73b": 285, "\uc990": 286, "\ubca4": 287, "\ube7c": 288, "\uc555": 289, "\ubca8": 290, "\ubcf5": 291, "\ub2a5": 292, "\uacf6": 293, "\ub860": 294, "\uc2b4": 295, "\ud4f0": 296, "\ucd18": 297, "\ud1b5": 298, "\uac78": 299, "\ud655": 300, "\uafd4": 301, "\ucc2c": 302, "\uc370": 303, "\uce98": 304, "\uaf42": 305, "\ub35c": 306, "\ub3d4": 307, "\uc77c": 308, "\ubb50": 309, "\uc6e0": 310, "\uac89": 311, "\uc7a6": 312, "\ubaa9": 313, "\ub2a0": 314, "\uc168": 315, "\ud568": 316, "\ucc59": 317, "\ub41c": 318, "\ub7fc": 319, "\uc0bf": 320, "\uc5ff": 321, "\uc22d": 322, "\uc220": 323, "\uaecd": 324, "\uca0c": 325, "\uc5bd": 326, "\uc3e0": 327, "\ub18d": 328, "\ub4f1": 329, "\uac71": 330, "\uce78": 331, "\uad76": 332, "\uccad": 333, "\uba3c": 334, "\ud769": 335, "\uc9d1": 336, "\ub2f3": 337, "\uc648": 338, "\ud56d": 339, "\ub8e1": 340, "\ub7f0": 341, "\ubd09": 342, "\uc1a1": 343, "\ub780": 344, "\ub525": 345, "\uc5c6": 346, "\uc0c1": 347, "\uccbc": 348, "\ub4dd": 349, "\ud508": 350, "\ubcc4": 351, "\uc53b": 352, "\uc270": 353, "\uacb9": 354, "\ubb47": 355, "\ud770": 356, "\ubed8": 357, "\ud0c4": 358, "\uc950": 359, "\uae0d": 360, "\ud3c9": 361, "\ucbe4": 362, "\uc559": 363, "\uc988": 364, "\ud314": 365, "\ub3c8": 366, "\uc5ed": 367, "\ub1cc": 368, "\uc9ec": 369, "\ub8f9": 370, "\ubd59": 371, "\uce59": 372, "\ud074": 373, "\uc654": 374, "\ub300": 375, "\ub3c5": 376, "\uac12": 377, "\ub904": 378, "\ud300": 379, "\ub9d1": 380, "\uc2ec": 381, "\ud654": 382, "\ub2cc": 383, "\uc9c8": 384, "\uc18d": 385, "\ub775": 386, "\ud138": 387, "\uccd0": 388, "\uc824": 389, "\ub69c": 390, "\ub053": 391, "\ud649": 392, "\uc73c": 393, "\ud749": 394, "\ucc3d": 395, "\ub784": 396, "\ub045": 397, "\ub7c9": 398, "\uaf41": 399, "\uae41": 400, "\ud1f4": 401, "\uacf3": 402, "\ud761": 403, "\ucc45": 404, "\ubab8": 405, "\uc548": 406, "\ud3a0": 407, "\uc989": 408, "\ub807": 409, "\uc6a9": 410, "\ub36e": 411, "\ub5a4": 412, "\ucc54": 413, "\ub480": 414, "\uba85": 415, "\uc0e4": 416, "\ub128": 417, "\ub140": 418, "\ub3d7": 419, "\ubc29": 420, "\uc54a": 421, "\uaca9": 422, "\ubf55": 423, "\ucc9c": 424, "\uce21": 425, "\uc92c": 426, "\uc2a4": 427, "\uba48": 428, "\uc52c": 429, "\uc528": 430, "\ud0b9": 431, "\uaf3d": 432, "\uac01": 433, "\uc12c": 434, "\ub9b4": 435, "\ud07c": 436, "\ub545": 437, "\ub204": 438, "\ud488": 439, "\ub454": 440, "\ubd10": 441, "\ucabd": 442, "\uce20": 443, "\ud759": 444, "\uce5c": 445, "\uc0b6": 446, "\uc9e7": 447, "\uc67c": 448, "\uc720": 449, "\ub460": 450, "\uaf08": 451, "\ube60": 452, "\ud639": 453, "\ud2bf": 454, "\ub010": 455, "\ub958": 456, "\ub3d9": 457, "\ub155": 458, "\uac14": 459, "\ub0c4": 460, "\ub9e1": 461, "\ub9b0": 462, "\uce89": 463, "\ucc0d": 464, "\ub54c": 465, "\uace1": 466, "\uc0ac": 467, "\uc6c3": 468, "\uadf8": 469, "\ud0d3": 470, "\uacb8": 471, "\ubc88": 472, "\uad0c": 473, "\ub2ed": 474, "\ub7fd": 475, "\ud2f8": 476, "\uc123": 477, "\uac15": 478, "\ud22c": 479, "\ub984": 480, "\uc728": 481, "\ubc97": 482, "\ud1a0": 483, "\uad90": 484, "\ub978": 485, "\ubaab": 486, "\ub78c": 487, "\uac08": 488, "\ubcd0": 489, "\ubc16": 490, "\ub95c": 491, "\uc219": 492, "\uc810": 493, "\ud569": 494, "\ub1a8": 495, "\ub7ab": 496, "\ubc8c": 497, "\ud78c": 498, "\uccb4": 499, "\uc88c": 500, "\uccab": 501, "\ub9e4": 502, "\uc131": 503, "\ud145": 504, "\ud1a1": 505, "\ub054": 506, "\ubcbd": 507, "\uacf1": 508, "\ub550": 509, "\uc5c7": 510, "\ucd5c": 511, "\uc5b9": 512, "\ube80": 513, "\ud2f0": 514, "\ube57": 515, "\ub9d0": 516, "\uc788": 517, "\ud39c": 518, "\uc564": 519, "\ud6e4": 520, "\ub20c": 521, "\uba74": 522, "\ub4ec": 523, "\ub791": 524, "\uc13c": 525, "\uc2dc": 526, "\uaca8": 527, "\ub611": 528, "\ud234": 529, "\ud3ec": 530, "\ubd48": 531, "\uc815": 532, "\ubcc0": 533, "\uc83c": 534, "\uc148": 535, "\ub9d9": 536, "\ubfcc": 537, "\uc0c0": 538, "\ub98e": 539, "\uc5d4": 540, "\ucf08": 541, "\uc800": 542, "\uacbc": 543, "\ub0d0": 544, "\uc7bd": 545, "\ud608": 546, "\uc838": 547, "\ud3fc": 548, "\ucf67": 549, "\uc717": 550, "\uc5e3": 551, "\ub098": 552, "\uc2e4": 553, "\uae00": 554, "\ucc60": 555, "\ub82c": 556, "\ucc2e": 557, "\uc369": 558, "\ucffc": 559, "\ub158": 560, "\uc9c1": 561, "\uc99d": 562, "\uc637": 563, "\ubd88": 564, "\uc878": 565, "\ud54f": 566, "\uc7a3": 567, "\ube14": 568, "\uc138": 569, "\uba78": 570, "\uc9d3": 571, "\ub2f7": 572, "\ubc84": 573, "\ub451": 574, "\ub0ab": 575, "\uc84c": 576, "\ud61c": 577, "\ube68": 578, "\uc918": 579, "\ud5cc": 580, "\uc124": 581, "\uc568": 582, "\ubcf4": 583, "\uac1c": 584, "\ub0ad": 585, "\ub4e4": 586, "\ub9e5": 587, "\ub07c": 588, "\ub0a9": 589, "\ub3c4": 590, "\ud143": 591, "\ub2ff": 592, "\uc2ed": 593, "\uaca0": 594, "\ubc40": 595, "\uc2b5": 596, "\uc0b4": 597, "\uc639": 598, "\uaf43": 599, "\ucad3": 600, "\ubc43": 601, "\uac77": 602, "\uc5ce": 603, "\ub110": 604, "\ub8cc": 605, "\ube5b": 606, "\uc80b": 607, "\uc885": 608, "\ucf8c": 609, "\uc30c": 610, "\uc57d": 611, "\ub7a8": 612, "\uc634": 613, "\ud551": 614, "\uac10": 615, "\uc557": 616, "\ub310": 617, "\ub141": 618, "\ud2b9": 619, "\ube8f": 620, "\uca54": 621, "\uc0d0": 622, "\ub918": 623, "\uae68": 624, "\ub099": 625, "\uc2fc": 626, "\ubc45": 627, "\uafc8": 628, "\ud3f0": 629, "\ucc3e": 630, "\ub429": 631, "\ub835": 632, "\ud47c": 633, "\ub531": 634, "\uc300": 635, "\uae61": 636, "\ub178": 637, "\uc3df": 638, "\uc090": 639, "\uc9c0": 640, "\uc0ec": 641, "\ub86f": 642, "\ud0d1": 643, "\uc27c": 644, "\ud751": 645, "\uc606": 646, "\uac90": 647, "\ub2c8": 648, "\ucca0": 649, "\uc313": 650, "\ud0f1": 651, "\ud33d": 652, "\ub0af": 653, "\uac80": 654, "\uc798": 655, "\uac04": 656, "\ub529": 657, "\uce90": 658, "\ubabb": 659, "\uc695": 660, "\uc2f6": 661, "\ub369": 662, "\uc1c4": 663, "\ub77c": 664, "\ub7ad": 665, "\ub834": 666, "\ud6e8": 667, "\ucc98": 668, "\ub4e0": 669, "\ub144": 670, "\uc2b7": 671, "\ud1b0": 672, "\ubd99": 673, "\ubc1d": 674, "\ub530": 675, "\ubabd": 676, "\uc2f8": 677, "\ud6a8": 678, "\ub97c": 679, "\uc598": 680, "\ud754": 681, "\ub8b0": 682, "\uc633": 683, "\ub0ae": 684, "\ub544": 685, "\ud63c": 686, "\uc625": 687, "\ud241": 688, "\uba70": 689, "\ud48d": 690, "\uc3dc": 691, "\uae54": 692, "\uaecf": 693, "\uac24": 694, "\ucd08": 695, "\ub5a8": 696, "\uaef4": 697, "\ub0a1": 698, "\uad04": 699, "\uc9d9": 700, "\ucc99": 701, "\uc9c4": 702, "\uba54": 703, "\uba39": 704, "\ub975": 705, "\ub05d": 706, "\uae08": 707, "\ubd04": 708, "\uc871": 709, "\uce35": 710, "\ud68c": 711, "\ud734": 712, "\ub790": 713, "\ubb38": 714, "\uc655": 715, "\ud1a8": 716, "\ud544": 717, "\uc190": 718, "\ud30c": 719, "\uc811": 720, "\ubb18": 721, "\uba40": 722, "\uce58": 723, "\uc30d": 724, "\uae34": 725, "\uc8e0": 726, "\ub9dd": 727, "\uc232": 728, "\ucfe8": 729, "\ud578": 730, "\uc6d0": 731, "\uce94": 732, "\ub418": 733, "\ucfc4": 734, "\uc158": 735, "\ub5b4": 736, "\uc721": 737, "\ub9cc": 738, "\uc55e": 739, "\uc50c": 740, "\uad00": 741, "\uc575": 742, "\uc801": 743, "\uc5f7": 744, "\ubf08": 745, "\ud38c": 746, "\ucd98": 747, "\ub744": 748, "\ud614": 749, "\uac11": 750, "\ub5bc": 751, "\ud55c": 752, "\ucc14": 753, "\ubb58": 754, "\ub4dc": 755, "\ub4e3": 756, "\uc4f8": 757, "\uc558": 758, "\ub2f4": 759, "\ud3b4": 760, "\ub450": 761, "\ub514": 762, "\ube44": 763, "\uba4d": 764, "\ubb34": 765, "\uc554": 766, "\ub3cb": 767, "\ud398": 768, "\uac81": 769, "\ud718": 770, "\uc54c": 771, "\ub00c": 772, "\ub154": 773, "\ucda4": 774, "\uc57c": 775, "\ucf30": 776, "\uc789": 777, "\uae0b": 778, "\uc069": 779, "\uac70": 780, "\ub044": 781, "\ud050": 782, "\ub6f0": 783, "\uc628": 784, "\ubca0": 785, "\ub0b4": 786, "\ud30d": 787, "\uc6c0": 788, "\ud0d0": 789, "\ub989": 790, "\ud131": 791, "\ubcbc": 792, "\ub0ac": 793, "\uc154": 794, "\ud560": 795, "\uc37c": 796, "\uc597": 797, "\ub0c5": 798, "\uac13": 799, "\ud5e8": 800, "\uc998": 801, "\uc0dd": 802, "\uac20": 803, "\uc5d0": 804, "\uad6c": 805, "\ub9e8": 806, "\ud14c": 807, "\uba87": 808, "\ub193": 809, "\uce30": 810, "\ub38c": 811, "\ucd2c": 812, "\uc5fd": 813, "\ub2ee": 814, "\ud5d8": 815, "\ubc0d": 816, "\uc778": 817, "\ud559": 818, "\ucfe0": 819, "\ubbc8": 820, "\ucd0c": 821, "\ub2d0": 822, "\uad7d": 823, "\uc5c9": 824, "\uaf5d": 825, "\uc5d1": 826, "\uae40": 827, "\ucabc": 828, "\ube59": 829, "\ucfe1": 830, "\ud750": 831, "\uc904": 832, "\uc27d": 833, "\ud384": 834, "\ub7ac": 835, "\ub85c": 836, "\uc21c": 837, "\ud3ab": 838, "\uc624": 839, "\ud53c": 840, "\ud0a4": 841, "\ub1fd": 842, "\uc77d": 843, "\uccb8": 844, "\ub960": 845, "\ub4ef": 846, "\ucf13": 847, "\uac74": 848, "\ucb49": 849, "\ucd1b": 850, "\uc874": 851, "\ud788": 852, "\ub824": 853, "\uc140": 854, "\uc88b": 855, "\ube75": 856, "\uc2f1": 857, "\ub9c9": 858, "\ub3cc": 860, "\ub0a0": 861, "\uc820": 862, "\ub465": 863, "\uc7b0": 864, "\uba5c": 865, "\uc549": 866, "\uba38": 867, "\ub561": 868, "\ucee5": 869, "\uc600": 870, "\uc587": 871, "\ud729": 872, "\ud6a1": 873, "\uc2a8": 874, "\uc9dc": 875, "\ub7ed": 876, "\ud65c": 877, "\uce74": 878, "\uc368": 879, "\uc740": 880, "\ub220": 881, "\ub385": 882, "\ud134": 883, "\ud0ac": 884, "\uc228": 885, "\uc544": 886, "\uc120": 887, "\ub72f": 888, "\uc538": 889, "\ub818": 890, "\ub9c1": 891, "\ud50c": 892, "\ub6ab": 893, "\uaf65": 894, "\ud1a4": 895, "\uc4f0": 896, "\uc81d": 897, "\uc724": 898, "\ud30e": 899, "\ub839": 900, "\ubaac": 901, "\ub2eb": 902, "\uacfd": 903, "\uc5f4": 904, "\uace8": 905, "\ud29c": 906, "\uc0d8": 907, "\ub3d5": 908, "\ud328": 909, "\ub108": 910, "\ud601": 911, "\uad18": 912, "\uc80a": 913, "\uadf9": 914, "\ud3d0": 915, "\uaff0": 916, "\uc553": 917, "\uacfc": 918, "\uc8c4": 919, "\uad8c": 920, "\uc678": 921, "\uc8fc": 922, "\uc0b0": 923, "\uc813": 924, "\ub2c9": 925, "\ubbf9": 926, "\uc758": 927, "\ub768": 928, "\ucd28": 929, "\ubbc0": 930, "\ub9ac": 931, "\uc3d8": 932, "\ubcf6": 933, "\ucc28": 934, "\ubc34": 935, "\ub800": 936, "\ub5a1": 937, "\uca5c": 938, "\ub825": 939, "\uc130": 940, "\ubd93": 941, "\ub4ed": 942, "\ucef4": 943, "\ucd95": 944, "\uc0f7": 945, "\ub8ec": 946, "\ub274": 947, "\ud14d": 948, "\ubc25": 949, "\ub0e5": 950, "\uaebc": 951, "\uc6b8": 952, "\uc9d5": 953, "\ud64d": 954, "\ub048": 955, "\ub0c8": 956, "\uc19f": 957, "\ub754": 958, "\ub798": 959, "\uafb8": 960, "\ud2c0": 961, "\ub2e4": 962, "\uc2dd": 963, "\ubcfc": 964, "\ub124": 965, "\uc1e0": 966, "\ub728": 967, "\uaf2d": 968, "\ud3bc": 969, "\ub2d8": 970, "\uc794": 971, "\uc7ac": 972, "\ub69d": 973, "\uc194": 974, "\uc5ec": 975, "\uc7c1": 976, "\uc0bd": 977, "\ube91": 978, "\ud758": 979, "\uac8c": 980, "\uae09": 981, "\ub560": 982, "\ud765": 983, "\ub51c": 984, "\ud4e8": 985, "\uacc1": 986, "\ud37c": 987, "\ub81b": 988, "\uce69": 989, "\ubdd4": 990, "\ud310": 991, "\uce68": 992, "\uc5b8": 993, "\uaf2c": 994, "\uae38": 995, "\uaed1": 996, "\ub374": 997, "\ucd09": 998, "\uc78e": 999, "\ud68d": 1000, "\uc790": 1001, "\ub518": 1002, "\uc911": 1003, "\ubc1b": 1004, "\ud640": 1005, "\uc81c": 1006, "\uaf3c": 1007, "\ud5f7": 1008, "\ucd94": 1009, "\ub080": 1010, "\ubc14": 1011, "\uacf5": 1012, "\uc560": 1013, "\ub354": 1014, "\uc501": 1015, "\ud648": 1016, "\uc561": 1017, "\ub5bb": 1018, "\uc2f9": 1019, "\uba58": 1020, "\uc2e0": 1021, "\ubb35": 1022, "\uc783": 1023, "\ub2aa": 1024, "\ud2ac": 1025, "\uae4e": 1026, "\ud479": 1027, "\ub420": 1028, "\ub837": 1029, "\ubbff": 1030, "\uc74c": 1031, "\uc2b9": 1032, "\ub86c": 1033, "\ub313": 1034, "\uac2f": 1035, "\ub96d": 1036, "\ucc0c": 1037, "\ucce4": 1038, "\ub85d": 1039, "\ud0c0": 1040, "\ud610": 1041, "\ub04a": 1042, "\ubf40": 1043, "\uc785": 1044, "\ub9bc": 1045, "\uc12f": 1046, "\ubc95": 1047, "\ub9de": 1048, "\ud600": 1049, "\uc288": 1050, "\uc719": 1051, "\ub7f4": 1052, "\ub0c7": 1053, "\uac1d": 1054, "\uc5fc": 1055, "\ub987": 1056, "\ucc29": 1057, "\uc70c": 1058, "\ub838": 1059, "\uc5b5": 1060, "\uc601": 1061, "\ud3f4": 1062, "\ub528": 1063, "\uca4d": 1064, "\uad34": 1065, "\uafbc": 1066, "\uc6e8": 1067, "\ubed0": 1068, "\ubc2d": 1069, "\ucca9": 1070, "\ub864": 1071, "\uc5bc": 1072, "\ub298": 1073, "\uc6b0": 1074, "\ud790": 1075, "\uc2eb": 1076, "\uc9dd": 1077, "\ucde8": 1078, "\ub428": 1079, "\ube5a": 1080, "\uc808": 1081, "\ubc18": 1082, "\uc5b4": 1083, "\ub3fc": 1084, "\ud638": 1085, "\ud760": 1086, "\ub9d8": 1087, "\uc1a5": 1088, "\ub79c": 1089, "\ub08c": 1090, "\ucea0": 1091, "\ubbf8": 1092, "\uc308": 1093, "\uae4c": 1094, "\uc149": 1095, "\ucf10": 1096, "\ub538": 1097, "\uc0c8": 1098, "\ucc30": 1099, "\uca4c": 1100, "\uc5c5": 1101, "\uc139": 1102, "\ub4c0": 1103, "\ub2e8": 1104, "\ub2d9": 1105, "\ubba4": 1106, "\ub150": 1107, "\ud034": 1108, "\ud56b": 1109, "\ud5ec": 1110, "\ub2f9": 1111, "\uc6d4": 1112, "\ucf65": 1113, "\ud480": 1114, "\ucd1d": 1115, "\uaddc": 1116, "\ub9ad": 1117, "\ubfdc": 1118, "\ub11b": 1119, "\ucf1c": 1120, "\uc6ec": 1121, "\uac19": 1122, "\uacac": 1123, "\ub192": 1124, "\uc05c": 1125, "\ub109": 1126, "\ud3b8": 1127, "\ub289": 1128, "\ub410": 1129, "\uc784": 1130, "\ucca8": 1131, "\uad70": 1132, "\uad75": 1133, "\uace7": 1134, "\uc6c5": 1135, "\ubdf0": 1136, "\uacc4": 1137, "\uc591": 1138, "\ud504": 1139, "\uc218": 1140, "\ud130": 1141, "\uc78a": 1142, "\ub840": 1143, "\ucee4": 1144, "\uc2ac": 1145, "\ub36b": 1146, "\uad74": 1147, "\ucef7": 1148, "\uc1fc": 1149, "\ub113": 1150, "\uafc0": 1151, "\ubc0b": 1152, "\uade0": 1153, "\ube8c": 1154, "\ub5c4": 1155, "\ubc94": 1156, "\uc14b": 1157, "\uc958": 1158, "\uafc7": 1159, "\ub118": 1160, "\ud32c": 1161, "\ubb44": 1162, "\uae65": 1163, "\ud0ec": 1164, "\ube10": 1165, "\uc6b1": 1166, "\uc7ad": 1167, "\ud31c": 1168, "\uc1e4": 1169, "\uc9da": 1170, "\ub3db": 1171, "\ud0dd": 1172, "\uc5bb": 1173, "\uc640": 1174, "\ud0d5": 1175, "\ubd95": 1176, "\ub828": 1177, "\ud558": 1178, "\ub808": 1179, "\uc11e": 1180, "\uc708": 1181, "\uc6c1": 1182, "\ub760": 1183, "\ucc3b": 1184, "\ud478": 1185, "\ud604": 1186, "\ubf51": 1187, "\ud48b": 1188, "\uc5d8": 1189, "\uc5e1": 1190, "\ub46c": 1191, "\uc880": 1192, "\uaf34": 1193, "\ub871": 1194, "\uc900": 1195, "\ud6c8": 1196, "\uc8fd": 1197, "\ucc4c": 1198, "\uacf0": 1199, "\uc20d": 1200, "\ud06c": 1201, "\ub367": 1202, "|": 859, "[UNK]": 1203, "[PAD]": 1204}

In [30]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [31]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

Load vocab.json file into tokenizer

In [32]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

loading file vocab.json
loading file tokenizer_config.json
loading file added_tokens.json
loading file special_tokens_map.json
Adding <s> to the vocabulary
Adding </s> to the vocabulary
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Prepare training data

In [34]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [35]:
fleurs_train = fleurs_train.map(prepare_dataset, remove_columns=fleurs_train.column_names)
fleurs_test = fleurs_test.map(prepare_dataset, remove_columns=fleurs_test.column_names)
fleurs_val = fleurs_val.map(prepare_dataset, remove_columns=fleurs_val.column_names)

Create repo and push tokenizer 

In [36]:
repo_name = "wav2vec2-large-xls-r-300m-kr-jw4169"
tokenizer.push_to_hub(repo_name)

tokenizer config file saved in wav2vec2-large-xls-r-300m-kr-jw4169/tokenizer_config.json
Special tokens file saved in wav2vec2-large-xls-r-300m-kr-jw4169/special_tokens_map.json
added tokens file saved in wav2vec2-large-xls-r-300m-kr-jw4169/added_tokens.json
Uploading the following files to jw4169/wav2vec2-large-xls-r-300m-kr-jw4169: tokenizer_config.json,vocab.json,special_tokens_map.json,added_tokens.json


CommitInfo(commit_url='https://huggingface.co/jw4169/wav2vec2-large-xls-r-300m-kr-jw4169/commit/21cc05aa2464033d68c7c26c529bac1060103878', commit_message='Upload tokenizer', commit_description='', oid='21cc05aa2464033d68c7c26c529bac1060103878', pr_url=None, pr_revision=None, pr_num=None)

Prepare for finetuning

In [37]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [38]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)



In [39]:
wer_metric = load_metric("wer")



In [40]:
import numpy as np
import random

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}



In [41]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

loading configuration file config.json from cache at /home/jw4169/.cache/huggingface/hub/models--facebook--wav2vec2-xls-r-300m/snapshots/1a640f32ac3e39899438a2931f9924c02f080a54/config.json
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extr

Freeze pretrained model

In [42]:
model.freeze_feature_extractor()


/opt/conda/lib/python3.7/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1638: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


Finetune model

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=4,
  gradient_accumulation_steps=4,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=False,
  save_steps=200,
  eval_steps=200,
  logging_steps=200,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=100,
  push_to_hub=True
)

In [50]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=fleurs_train,
    eval_dataset=fleurs_val,
    tokenizer=processor.feature_extractor,
)

/home/jw4169/wav2vec2-large-xls-r-300m-kr-jw4169 is already a clone of https://huggingface.co/jw4169/wav2vec2-large-xls-r-300m-kr-jw4169. Make sure you pull the latest changes with `repo.git_pull()`.


In [58]:
import gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

trainer.train(resume_from_checkpoint='wav2vec2-large-xls-r-300m-kr-jw4169/checkpoint-1800')
trainer.push_to_hub()

Loading model from wav2vec2-large-xls-r-300m-kr-jw4169/checkpoint-1800.
***** Running training *****
  Num examples = 2307
  Num Epochs = 30
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 4320
  Number of trainable parameters = 312465719
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 12
  Continuing training from global step 1800
  Will skip the first 12 epochs then the first 288 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/288 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "


Step,Training Loss,Validation Loss,Wer
2000,0.271200,0.894803,0.585402
2200,0.228300,0.900942,0.549506
2400,0.182500,0.907857,0.550105
2600,0.161000,0.951825,0.539037
2800,0.139400,0.952889,0.539934
3000,0.126600,0.950529,0.528268
3200,0.110200,0.974792,0.532755
3400,0.101000,0.959270,0.531558
3600,0.090700,0.983154,0.529165
3800,0.083300,0.977310,0.518098


***** Running Evaluation *****
  Num examples = 226
  Batch size = 8
Saving model checkpoint to wav2vec2-large-xls-r-300m-kr-jw4169/checkpoint-2000
Configuration saved in wav2vec2-large-xls-r-300m-kr-jw4169/checkpoint-2000/config.json
Model weights saved in wav2vec2-large-xls-r-300m-kr-jw4169/checkpoint-2000/pytorch_model.bin
Feature extractor saved in wav2vec2-large-xls-r-300m-kr-jw4169/checkpoint-2000/preprocessor_config.json
Feature extractor saved in wav2vec2-large-xls-r-300m-kr-jw4169/preprocessor_config.json
/opt/conda/lib/python3.7/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "
***** Running Evaluation **

Upload file pytorch_model.bin:   0%|          | 4.00k/1.18G [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/jw4169/wav2vec2-large-xls-r-300m-kr-jw4169
   c7d1e6a..0f86ee4  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/jw4169/wav2vec2-large-xls-r-300m-kr-jw4169
   c7d1e6a..0f86ee4  main -> main

To https://huggingface.co/jw4169/wav2vec2-large-xls-r-300m-kr-jw4169
   0f86ee4..0fadad7  main -> main

   0f86ee4..0fadad7  main -> main



'https://huggingface.co/jw4169/wav2vec2-large-xls-r-300m-kr-jw4169/commit/0f86ee41b3314b1b9f6b98e043873c14335770f4'